In [1]:
!pip install flask_ngrok

In [2]:
import numpy as np
import pandas as pd
from flask import Flask, request, render_template, current_app
from flask_ngrok import run_with_ngrok

# Modeling libs
from gensim.models import Word2Vec

# Others
import json
import os
from google.colab import files
from google.colab import drive

In [3]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 5.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=4fb7bcecdc5842339bfd8c9180a423b9dafca6fb662c8907a1ceef39e27ff487
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [4]:
! ngrok authtoken 208sLlAgTDc0ok3qVPgNrIZQgwD_4vSfQH6Bcn68Y3QAND4j1

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd /content/drive/MyDrive/School [ALU]/Computer Science/Y_4/TERM2/Data Mining/CBF_REC/deploy

/content/drive/MyDrive/School [ALU]/Computer Science/Y_4/TERM2/Data Mining/CBF_REC/deploy


In [7]:
app = Flask(__name__)
model = Word2Vec.load('/content/drive/MyDrive/School [ALU]/Computer Science/Y_4/TERM2/Data Mining/CBF_REC/deploy/song2vec.model')

In [8]:
# import dataframe of songs
songs = pd.read_csv('/content/drive/MyDrive/School [ALU]/Computer Science/Y_4/TERM2/Data Mining/CBF_REC/deploy/static/data/limited_songs.csv')

In [9]:
songs.set_index('song_id', inplace=True)
songs.shape

(16594, 3)

In [10]:
run_with_ngrok(app)
@app.route('/')
def home():
    return render_template('index.html')

In [11]:
@app.route('/getrecommendation',methods=['POST'])
def getrecommendation():
    file = request.files['fileup']
    
    file_name = file.filename
    file.save(file_name)
    
    with open(file_name) as json_file:
        playlist_test = json.load(json_file)
        
    playlist_vec = list(map(meanVectors, playlist_test))
    recommendation, input_playlist = print_recommended_songs(0, 10, playlist_test, playlist_vec)

    return render_template('results.html', output=recommendation, input_play=input_playlist)

In [12]:
def meanVectors(playlist):
    vec = []
    for song_id in playlist:
        try:
            vec.append(model.wv[song_id])
        except KeyError:
            continue
    return np.mean(vec, axis=0)

In [13]:
def similarSongsByVector(vec, n = 10, by_name = True):
    # extract most similar songs for the input vector
    similar_songs = model.wv.similar_by_vector(vec, topn = n)
    
    # extract name and similarity score of the similar products
    if by_name:
        similar_songs = [(songs.loc[int(song_id), "artist - title"], sim)
                              for song_id, sim in similar_songs]
    
    return similar_songs

In [14]:
def print_recommended_songs(idx, n, playlist_test, vec):
    
    rec_playlist = []
    input_playlist = []
    
    for song_id in playlist_test[idx]:
        song_id = int(song_id)
        input_playlist.append(songs.loc[song_id, "artist - title"])
    
    for song, sim in similarSongsByVector(vec[idx], n):
        rec_playlist.append(song)
        
    return rec_playlist, input_playlist

In [17]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ba38-35-243-193-65.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [29/Apr/2022 13:26:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 13:26:45] "GET /static/js/script.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 13:26:45] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 13:26:46] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Apr/2022 13:27:00] "POST /getrecommendation HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 13:37:05] "GET / HTTP/1.1" 200 -
